## Importing the libraries

In [ ]:
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime
from urllib.request import urlopen

## Scraping the initial data with BeautifulSoup

In [ ]:
URL = "https://www.etuovi.com/myytavat-asunnot/helsinki?haku=M1608807886&sivu=1"
#requesting the URL above:
page = requests.get(URL)
#specifying the format of “page” using the html parser allowimg python to read the various components of the page, rather than treating it as one long string.
soup = bs(page.text, "html.parser")
#printing soup in a priettier form:
print(soup.prettify())

In [ ]:
# Creating a while loop for looping through all the "next pages".
i=0
initial_data = []
while True:
    i = i+1
    #Edit the if function below to scrape a larger number of pages
    if i > 6:
        print("done")
        break
    else:
        #Edit the initial search criteria in at etuovi.com for more specified search.
        url = f"https://www.etuovi.com/myytavat-asunnot/helsinki?haku=M1608933110&sivu="+str(i)
        page = requests.get(url)
        soup = bs(page.text, "html.parser")
        for div in soup.find_all(name="div", attrs={"class":"flexboxgrid__col-xs-12__1I1LS flexboxgrid__col-sm-7__1EzZq flexboxgrid__col-md-9__2kjy7 flexboxgrid__col-lg-9__M7bfm styles__infoArea__2yhEL"}):
            initial_data.append(div.text.strip())
        print(initial_data)

## Cleaning the data

In [ ]:
initial = pd.DataFrame(initial_data)
df_initial = initial.rename(columns={0: 'Raw_Data'})

In [ ]:
# Clean the initial soup into separated columns for easier readibility and management
df_initial[['Type','Raw_2']] = df_initial.Raw_Data.str.split("|",expand=True)
df_initial[["Raw_3", "Year"]] = df_initial.Raw_2.str.split("Vuosi", expand=True)
df_initial['Year_Built'] = df_initial['Year'].str[:4]
df_initial[["Raw_4", "Price_Iteration"]] = df_initial.Raw_3.str.split("Hinta", expand=True)
df_initial[["Price", "Size"]] = df_initial.Price_Iteration.str.split("Koko", expand=True)
df_initial[["Rooms","Address"]] = df_initial.Raw_3.str.split("check", expand=True)
df_initial['Price_Iteration_2'] = df_initial['Price'].str.replace('*',"")
df_initial['Price_Iteration_3'] = df_initial['Price_Iteration_2'].str.replace('€',"")
df_initial['Price_Iteration_4'] = df_initial['Price_Iteration_3'].str[:7]
df_initial['Price_euro'] = df_initial['Price_Iteration_4'].str.replace("\s+","")
df_initial["Size_m²"] = df_initial["Size"].str.replace(" m²","")
print(df_initial.head())

In [ ]:
# save the processed colums, rearrange the columns, and get rid of the raw data
data_cleansed = df_initial[["Price_euro","Year_Built","Size_m²","Type","Rooms","Address"]]

## Saving the dataframe to Excel

In [ ]:
#add a datetime for naming the excel sheet
dt = datetime.now(tz=None)
dt_str = dt.strftime("%Y" + "%m" + "%d")

In [ ]:
# save the data into Excel (change the file path)
data_cleansed.to_excel(r'C:\Users\JohnSmith\NeighborhoodWatch.xlsx', index = False, sheet_name=dt_str)